In [ ]:
import sys
sys.path.insert(0, '..')

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({'font.size': 16})

import numpy as np

from scipy.optimize import curve_fit

from astropy.io import fits

import reference_LFs as refLF
from paus_utils import *

from jpasLAEs.utils import bin_centers
from load_lya_LF import load_combined_LF

In [ ]:
def power_fit(Lx, A, B):
    return 10 ** (A * (np.log10(Lx) - 43.5) + B)
    
def load_mcmc_powerlaw_fit():
    flat_samples = np.load('/home/alberto/cosmos/LAEs/npy/mcmc_powerlaw_fit_chain.npy')

    Phi_fit_i = []
    for ii, step in enumerate(flat_samples[::-1]):
        if ii == 10_000:
            break
        Phi_fit_i.append(power_fit(Lx, step[0], step[1]))
    Phi_fit_84 = np.percentile(Phi_fit_i, 84, axis=0)
    Phi_fit_16 = np.percentile(Phi_fit_i, 16, axis=0)
    Phi_fit_50 = np.percentile(Phi_fit_i, 50, axis=0)
    return Phi_fit_16, Phi_fit_50, Phi_fit_84

In [ ]:
# Load the QSO mock to compare
from load_paus_mocks import load_qso_mock
from jpasLAEs.utils import flux_to_mag

source_cats_dir = '/home/alberto/almacen/Source_cats'
mock_path = f'{source_cats_dir}/QSO_PAUS_LAES_2'
mock = load_qso_mock(mock_path)

mock['r_mag'] = flux_to_mag(mock['flx_0'][-4], w_central[-4])

# Load the W3 catalog Xmatch with SDSS

# sdss_xm = pd.read_csv('/home/alberto/almacen/PAUS_data/catalogs/Xmatch_SDSS_W3.csv')

In [ ]:
from curve_fit.schechter_fit import schechter
from curve_fit.dpl_fit_UV import double_power_law

def load_mcmc_schechter_fit(nb1, nb2, suffix=''):
    flat_samples = np.load(f'../curve_fit/chains/mcmc_schechter_fit_chain_nb{nb1}-{nb2}{suffix}.npy')

    Phi_fit_i = []
    for ii, step in enumerate(flat_samples[::-1]):
        if ii == 1_000:
            break

        this_alpha = step[2]
        this_Lstar = step[1]
        this_Phistar = step[0]

        if suffix == '_fixed_alpha':
            this_alpha = -1.63
        if suffix == '_fixed_Phistar':
            this_Phistar = -6.721022310205154
        if suffix == '_fixed_Lstar':
            this_Lstar = 44.95

        Phi_fit_i.append(schechter(Lx, 10**this_Phistar, 10**this_Lstar, this_alpha)
                        * Lx * np.log(10))
    Phi_fit_84 = np.percentile(Phi_fit_i, 84, axis=0)
    Phi_fit_16 = np.percentile(Phi_fit_i, 16, axis=0)
    Phi_fit_50 = np.percentile(Phi_fit_i, 50, axis=0)
    Phi_fit_05 = np.percentile(Phi_fit_i, 5, axis=0)
    Phi_fit_95 = np.percentile(Phi_fit_i, 95, axis=0)
    return Phi_fit_16, Phi_fit_50, Phi_fit_84, Phi_fit_05, Phi_fit_95

def load_mcmc_dpl_fit(nb1, nb2, suffix=''):
    flat_samples = np.load(f'../curve_fit/chains/mcmc_UV_dpl_fit_chain_nb{nb1}-{nb2}{suffix}.npy')


    Phi_fit_i = []
    for ii, step in enumerate(flat_samples[::-1]):
        if ii == 10_000:
            break

        this_Phistar = step[0]
        this_Mbreak = step[1]
        this_beta = step[2]
        this_gamma = step[3]

        if suffix == '_fixed_gamma':
            this_gamma = 4.5
        if suffix == '_fixed_beta':
            this_beta = 1.25

        Phi_fit_i.append(double_power_law(M_Arr, this_Phistar, this_Mbreak, this_beta, this_gamma))

    Phi_fit_84 = np.percentile(Phi_fit_i, 84, axis=0)
    Phi_fit_16 = np.percentile(Phi_fit_i, 16, axis=0)
    Phi_fit_50 = np.percentile(Phi_fit_i, 50, axis=0)
    Phi_fit_05 = np.percentile(Phi_fit_i, 5, axis=0)
    Phi_fit_95 = np.percentile(Phi_fit_i, 95, axis=0)
    return Phi_fit_16, Phi_fit_50, Phi_fit_84, Phi_fit_05, Phi_fit_95

In [ ]:
# survey_list = ['W3', 'W1', 'W2']
# nb_list = [[0, 3], [2, 5], [4, 7], [6, 9], [8, 11], [10, 13], [12, 15], [14, 18]]


# LyaLF = load_combined_LF(survey_list, nb_list, combined_LF=True)

# # # LF of the mock
# # nb1, nb2 = nb_list[0][0], nb_list[-1][-1]
# # L_bins = np.linspace(40, 47, 100)
# # L_bins_c = bin_centers(L_bins)
# # L_bins_w = L_bins[1] - L_bins[0]
# # mask_mock = ((NB_z(mock['zspec']) >= nb1)
# #              & (NB_z(mock['zspec']) <= nb2)
# #              & (mock['EW0_lya_spec'] > 30)
# #              & (mock['r_mag'] < 24)
# #              & (mock['r_mag'] > 17))
# # mock_LF = np.histogram(mock['L_lya_spec'][mask_mock], L_bins)[0]\
# #               / L_bins_w / Lya_effective_volume(nb1, nb2, 400)
# # ax.plot(L_bins_c, mock_LF)

# fig, ax = plt.subplots(figsize=(6, 4))

# [yerr_up, yerr_down] = LyaLF['LF_total_err']
# # yerr_up = (yerr_up**2 + LyaLF['poisson_err']**2)**0.5
# # yerr_down = (yerr_down**2 + LyaLF['poisson_err']**2)**0.5

# # ax.errorbar(LyaLF['LF_bins'], LyaLF['LF_total'],
# #             yerr=[yerr_down, yerr_up],
# #             fmt='s', ls='', mfc='none', mec='k',
# #             ms=7, ecolor='k', capsize=3)
# mask_LF_bins = (LyaLF['LF_bins'] >= 40)
# ax.errorbar(LyaLF['LF_bins'][mask_LF_bins], LyaLF['LF_boots'][mask_LF_bins],
#             yerr=[yerr_down[mask_LF_bins], yerr_up[mask_LF_bins]],
#             fmt='s', ls='', mfc='limegreen', mec='k',
#             ms=7, ecolor='k', capsize=3,
#             label=f'This work $(z={z_NB(nb_list[0][0])-0.05:0.2f}-{z_NB(nb_list[-1][-1])+0.05:0.2f})$')

# reference_LFs = [refLF.l22, refLF.z21, refLF.tt23_z2038]
# for j, lf in enumerate(reference_LFs):
#     ax.errorbar(lf['logL'], lf['Phi'],
#                 yerr=[lf['yerr_minus'], lf['yerr_plus']],
#                 linestyle='', fmt=lf['fmt'], c=lf['color'],
#                 alpha=0.9, label=lf['label'], capsize=3,
#                 zorder=-99)

# # My fit
# Lx = np.logspace(42, 46.5, 1000)
# Phi_fit_16, Phi_fit_50, Phi_fit_84, Phi_fit_05, Phi_fit_95 = load_mcmc_schechter_fit(0, 18)

# ax.plot(np.log10(Lx), Phi_fit_50,
#         c='limegreen')
# # ax.fill_between(np.log10(Lx), Phi_fit_05, Phi_fit_95,
# #                 alpha=0.3, color='C0', lw=0)
# ax.fill_between(np.log10(Lx), Phi_fit_16, Phi_fit_84,
#                 alpha=0.25, color='limegreen', lw=0)

# # Save this fit to plot it later as reference
# total_LF_bins = np.log10(Lx)
# total_LF_Phi = Phi_fit_50

# ax.set_xticks(np.arange(42, 47, 0.5))
# ax.set_yticks(np.logspace(-10, -1, 7))

# ax.set_yscale('log')
# ax.set_ylim(1e-10, 1e-1)
# ax.set_xlim(42.4, 46.1)

# ax.legend(fontsize=11)

# ax.set_xlabel(r'$\log_{10}(L_{\mathrm{Ly}\alpha}/\mathrm{erg\,s}^{-1})$')
# ax.set_ylabel(r'$\Phi$ [Mpc$^{-3}\,\Delta\log_{10}L^{-1}$]')


# fig.savefig('../figures/total_LF.pdf', bbox_inches='tight', pad_inches=0.1,
#             facecolor='w')
# plt.show()

In [ ]:
survey_list = ['W3', 'W1', 'W2']
nb_list = [[0, 3], [2, 5], [4, 7], [6, 9], [8, 11], [10, 13], [12, 15], [14, 18]]


LyaLF = load_combined_LF(survey_list, nb_list, combined_LF=True, LF_kind='UV', merge_bins=True)


fig, ax = plt.subplots(figsize=(6, 4))

[yerr_up, yerr_down] = LyaLF['LF_total_err']
yerr_up = (yerr_up**2 + LyaLF['poisson_err']**2)**0.5
yerr_down = (yerr_down**2 + LyaLF['poisson_err']**2)**0.5

mask_LF_bins = (LyaLF['LF_bins'] < 40)
ax.errorbar(LyaLF['LF_bins'][mask_LF_bins], LyaLF['LF_boots'][mask_LF_bins],
            yerr=[yerr_down[mask_LF_bins], yerr_up[mask_LF_bins]],
            fmt='s', ls='', mfc='limegreen', mec='k',
            ms=7, ecolor='k', capsize=3,
            label=f'This work $(z={z_NB(nb_list[0][0])-0.05:0.2f}-{z_NB(nb_list[-1][-1])+0.05:0.2f})$')

# My fit
Lx = np.linspace(-30, -15, 1000)
M_Arr = Lx
Phi_fit_16, Phi_fit_50, Phi_fit_84, Phi_fit_05, Phi_fit_95 = load_mcmc_dpl_fit(0, 18)

ax.plot(np.log10(Lx), Phi_fit_50,
        c='limegreen')
ax.fill_between(np.log10(Lx), Phi_fit_16, Phi_fit_84,
                alpha=0.25, color='limegreen', lw=0)


ax.set_ylim(1e-9, 1e-5)
ax.set_xlim(-30, -20)
ax.set_yscale('log')
ax.invert_xaxis()

ax.tick_params(labelsize=17, direction='in', which='both')
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')

ax.set_ylabel(r'$\Phi$ [Mpc$^{-3}\,\Delta$mag$^{-1}$]', fontsize=20)
ax.set_xlabel(r'$M_{{\rm UV, 1450}}$ [mag]',
    fontsize=20)



fig.savefig('../figures/total_LF_UV.pdf', bbox_inches='tight', pad_inches=0.1,
            facecolor='w')
plt.show()

In [ ]:
from plot.puricomp1D import plot_puricomp1d

r_min, r_max = 17, 24
puricomp1d_L_bins = np.linspace(-31, -20, 10)
puricomp1d_L_bins_c = bin_centers(puricomp1d_L_bins)

puri1d, comp1d = plot_puricomp1d('W1', 0, 18,
                                    r_min, r_max,
                                    L_bins=puricomp1d_L_bins,
                                    LF_kind='UV')

puricomp1d_L_bins_lya = np.linspace(42.5, 45.5, 15)
puricomp1d_L_bins_c_lya = bin_centers(puricomp1d_L_bins)
puri1d_lya, comp1d_lya = plot_puricomp1d('W1', 0, 18,
                                    r_min, r_max,
                                    L_bins=puricomp1d_L_bins,
                                    LF_kind='Lya')

MUV_Arr_hiz = [-25.8203611435952, -26.212678406852586, -24.740548987801873, -24.874748293431768, -25.174493132247367, -25.602635160990275, -23.89222607237893, -23.9203479579364, -24.690615455415514, -24.7771626754886, -24.15656403384465, -25.09473542804817, -24.660957929355966, -27.48978954114158, -25.63107733857017, -27.11613266030886, -25.009457289541547, -24.879800171402128]
MUV_e_Arr = [0.11818163122030359, 0.029330895423108582, 0.21122928347527536, 0.18454288586918452, 0.16338634354078094, 0.04101937457377991, 0.13203216347706037, 0.12600382326959442, 0.17985356707774844, 0.22956735374987605, 0.4327381891239168, 0.25491923639678593, 0.1842595221993075, 0.040392473665365, 0.10596378281378038, 0.0313305870083058, 0.17442288222306845, 0.21355673942153533]

In [ ]:
survey_list = ['W3', 'W1', 'W2']
# nb_list = [[0, 2], [2, 4], [4, 6], [6, 8],
#            [8, 10], [10, 12], [12, 14], [14, 16], [16, 18]]
nb_list = [[0, 3], [2, 5], [4, 7], [6, 9], [8, 11], [10, 13], [12, 15], [14, 18]]

fig, axs = plt.subplots(3, 3, figsize=(14, 14), sharey=True, sharex=True,
                        width_ratios=[1, 1, 1], height_ratios=[1, 1, 1])

reference_LFs = [[refLF.tt23_z2630, refLF.s18d],
                 [refLF.tt23_z2833, refLF.u08, refLF.s18b],
                 [refLF.tt23_z2833, refLF.u08, refLF.s18c, refLF.ds20_z324,
                  refLF.l22_z324],
                 [refLF.tt23_z3135, refLF.s18e, refLF.ds20_z324, refLF.l22_z324],
                 [refLF.tt23_z3338, refLF.s18f],
                 [refLF.tt23_z3338, refLF.s18f],
                 [],
                 [],
                 []]

chi2 = []
chi2_fa = []
chi2_fp = []
chi2_fl = []

LFs_mult_list_combi = []
LFs_err_mult_list_combi = []

for iii, ax in enumerate(axs.flatten()):
    if iii < 8:
        LyaLF = load_combined_LF(survey_list, [nb_list[iii]], merge_bins=True)


        [yerr_up, yerr_down] = LyaLF['LF_total_err']
        yerr_up = (yerr_up**2 + LyaLF['poisson_err']**2)**0.5
        yerr_down = (yerr_down**2 + LyaLF['poisson_err']**2)**0.5
        # ax.errorbar(LyaLF['LF_bins'] + 0.02, LyaLF['LF_total'],
        #             yerr=[yerr_down, yerr_up],
        #             fmt='s', ls='', mfc='none', mec='sienna',
        #             ms=6, ecolor='sienna', capsize=3)
        mask_LF_bins = (LyaLF['LF_bins'] < 45.5)
        ax.errorbar(LyaLF['LF_bins'][mask_LF_bins], LyaLF['LF_boots'][mask_LF_bins],
                    yerr=[yerr_down[mask_LF_bins], yerr_up[mask_LF_bins]],
                    fmt='s', ls='', mfc='limegreen', mec='k',
                    ms=6, ecolor='k', capsize=3,
                    label='This work', zorder=99)

        LFs_mult_list_combi.append(LyaLF['LF_boots'])
        LFs_err_mult_list_combi.append([yerr_up, yerr_down])

        nb_min = nb_list[iii][0]
        nb_max = nb_list[iii][1]
    else:
        ## For the last bin
        vi_cat_hiz = fits.open('/home/alberto/almacen/PAUS_data/catalogs/LAE_selection_VI_hiZ.fits')[1].data
        hiz_mask = vi_cat_hiz['is_hiZ_LAE']

        L_min, L_max = 40, 47
        N_bins = 20
        L_bins = np.linspace(L_min, L_max, N_bins + 1)

        nb_min = vi_cat_hiz['lya_NB'][hiz_mask].min()
        nb_max = vi_cat_hiz['lya_NB_VI'][hiz_mask].max()

        vol_hiz = 0.
        for field_name in ['W1', 'W2', 'W3']:
            vol_hiz += Lya_effective_volume(nb_min, nb_max, field_name)

        L_Arr_hiz = vi_cat_hiz['L_lya_corr'][hiz_mask]
        L_e_Arr = [vi_cat_hiz['L_lya_corr_err_down'][hiz_mask], vi_cat_hiz['L_lya_corr_err_up'][hiz_mask]]
        LF_mat_hiz = []
        for jjj in range(100):
            L_Arr = np.random.choice(L_Arr_hiz, len(L_Arr_hiz), replace=True)

            randN = np.random.randn(len(L_Arr))
            L_perturbed = np.empty_like(L_Arr)
            L_perturbed[randN <= 0] = (L_Arr + L_e_Arr[0] * randN)[randN <= 0]
            L_perturbed[randN > 0] = (L_Arr + L_e_Arr[1] * randN)[randN > 0]
            L_perturbed[np.isnan(L_perturbed)] = 0.

            this_puri = np.interp(L_perturbed, puricomp1d_L_bins_c_lya, puri1d_lya)
            this_comp_sel = np.interp(L_perturbed, puricomp1d_L_bins_c_lya, comp1d_lya)
            weights = 1. / this_comp_sel

            LF_mat_hiz.append(np.histogram(L_perturbed, L_bins)[0] / (L_bins[1] - L_bins[0]) / vol_hiz)

        LF_hiz = np.mean(LF_mat_hiz, axis=0)
        LF_hiz_std = np.std(LF_mat_hiz, axis=0)
        LF_hiz_err_down = LF_hiz - np.percentile(LF_mat_hiz, axis=0, q=16)
        LF_hiz_err_up = np.percentile(LF_mat_hiz, axis=0, q=84) - LF_hiz

        mask_LF_bins = (bin_centers(L_bins) >= 43.5)
        ax.errorbar(bin_centers(L_bins)[mask_LF_bins], LF_hiz[mask_LF_bins],
                    # yerr=[LF_hiz_err_down[mask_LF_bins], LF_hiz_err_up[mask_LF_bins]],
                    yerr=LF_hiz_std[mask_LF_bins],
                    fmt='s', ls='', mfc='none', mec='k',
                    ms=6, ecolor='k', capsize=3,
                    label='This work', lolims=False, zorder=99)


    if not iii % axs.shape[1]:
        ax.set_ylabel(r'$\Phi$ [Mpc$^{-3}\,\Delta\log_{10}L^{-1}$]', fontsize=20)
    if iii >= len(axs.flatten()) - axs.shape[1]:
        ax.set_xlabel(r'$\log_{10}(L_{\mathrm{Ly}\alpha}/\mathrm{erg\,s}^{-1})$',
                      fontsize=20)


    z_min = (w_central[nb_min] - fwhm_Arr[nb_min] * 0.5) / w_lya - 1
    z_max = (w_central[nb_max] + fwhm_Arr[nb_max] * 0.5) / w_lya - 1
    ax.text(42.9, 7e-4, fr'\bf z = {z_min:0.2f}--{z_max:0.2f}',
            fontsize=13)

    for j, lf in enumerate(reference_LFs[iii]):
        ax.errorbar(lf['logL'], lf['Phi'],
                    yerr=[lf['yerr_minus'], lf['yerr_plus']],
                    linestyle='', fmt=lf['fmt'], c=lf['color'],
                    alpha=1, label=lf['label'], capsize=3,
                    zorder=-99)

    # Schechter fit
    if iii < 8:
        Lx = np.logspace(42, 46, 2000)
        Phi_fit_16, Phi_fit_50, Phi_fit_84, Phi_fit_05, Phi_fit_95 =\
            load_mcmc_schechter_fit(*nb_list[iii])

        ax.plot(np.log10(Lx), Phi_fit_50,
                c='limegreen', zorder=98, ls='-', lw=2)
        ax.fill_between(np.log10(Lx), Phi_fit_16, Phi_fit_84,
                        alpha=0.25, color='limegreen', lw=0)
        this_chi2 = (LyaLF['LF_total'] - np.interp(LyaLF['LF_bins'], Lx, Phi_fit_50))**2 / ((yerr_up - yerr_down)*0.5) ** 2
        chi2.append(np.log10(np.sum(this_chi2[np.isfinite(this_chi2) & mask_LF_bins])))

        Phi_fit_16, Phi_fit_50, Phi_fit_84, Phi_fit_05, Phi_fit_95 =\
            load_mcmc_schechter_fit(*nb_list[iii], suffix='_fixed_alpha')
        this_chi2 = (LyaLF['LF_total'] - np.interp(LyaLF['LF_bins'], Lx, Phi_fit_50))**2 / ((yerr_up - yerr_down)*0.5) ** 2
        chi2_fa.append(np.log10(np.sum(this_chi2[np.isfinite(this_chi2) & mask_LF_bins])))

        ax.plot(np.log10(Lx), Phi_fit_50,
                c='crimson', zorder=97, ls='-.', lw=2)

        Phi_fit_16, Phi_fit_50, Phi_fit_84, Phi_fit_05, Phi_fit_95 =\
            load_mcmc_schechter_fit(*nb_list[iii], suffix='_fixed_Lstar')
        this_chi2 = (LyaLF['LF_total'] - np.interp(LyaLF['LF_bins'], Lx, Phi_fit_50))**2 / ((yerr_up - yerr_down)*0.5) ** 2
        chi2_fl.append(np.log10(np.sum(this_chi2[np.isfinite(this_chi2) & mask_LF_bins])))

        ax.plot(np.log10(Lx), Phi_fit_50,
                c='teal', zorder=96, ls='--', lw=2)
    if iii == 8:
        Lx = np.logspace(42, 46, 2000)
        Phi_fit_16, Phi_fit_50, Phi_fit_84, Phi_fit_05, Phi_fit_95 =\
            load_mcmc_schechter_fit(nb_min, nb_max)

        # ax.plot(np.log10(Lx), Phi_fit_50,
        #         c='limegreen', ls='-', lw=2, zorder=95)
        # ax.fill_between(np.log10(Lx), Phi_fit_16, Phi_fit_84,
        #                 alpha=0.25, color='limegreen', lw=0, ls='--', lw=2)

    # Plot the total LF as reference
    # ax.plot(LyaLF['LF_bins'], total_LF_Phi, c='k', ls=':', zorder=-90)


    ax.tick_params(labelsize=17, direction='in', which='both')
    ax.set_yticks(np.logspace(-10, -3, 8))
    xticks = np.arange(43, 46, 0.5)
    ax.set_xticks(xticks)
    ax.yaxis.set_ticks_position('both')
    ax.xaxis.set_ticks_position('both')

    ax.set_ylim(1e-9, 2e-3)
    ax.set_xlim(42.7, 45.9)
    ax.set_yscale('log')


    ax.legend(fontsize=10)



fig.subplots_adjust(wspace=0.02, hspace=0.02)

fig.savefig('../figures/multiLF.pdf', bbox_inches='tight', pad_inches=0.1,
            facecolor='w')
plt.show()

In [ ]:
from astropy.cosmology import LambdaCDM
cosmo_ref = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7)

In [ ]:
survey_list = ['W3', 'W1', 'W2']
nb_list = [[0, 3], [2, 5], [4, 7], [6, 9], [8, 11], [10, 13], [12, 15], [14, 18]]

fig, axs = plt.subplots(3, 3, figsize=(14, 14), sharey=True, sharex=True,
                        width_ratios=[1, 1, 1], height_ratios=[1, 1, 1])

# Reference LFs Kulkarni+19
k19_ref_MUV_LF = fits.open('~/almacen/AGN_UV_LF_literature_data/AGN_UV_LF_Kulkarni2019.fits')[3].data
ref_row_idx = [[13, 14], [15, 16, 17], [17, 18, 19],
               [18, 19, 20], 20, [20], [20, 21], [21], [22]]

## Visual inspection cat
vi_cat_hiz_uv = fits.open('/home/alberto/almacen/PAUS_data/catalogs/PAUS_LAE_selection_visual_insp_AT.fits')[1].data
vi_cat_hiz = fits.open('/home/alberto/almacen/PAUS_data/catalogs/LAE_selection_VI_hiZ.fits')[1].data
# hiz_mask = np.ones_like(vi_cat_hiz_uv['is_LAE_VI']).astype(bool)
hiz_mask = vi_cat_hiz_uv['is_LAE_VI']

# Liu+22 M_UV LF
L22_UV = pd.read_csv('/data/alberto/PAUS_data/literature_UV_LFs/Liu_M_UV_z2.6.csv')

for iii, ax in enumerate(axs.flatten()):
    if iii < 8:
        LyaLF = load_combined_LF(survey_list, [nb_list[iii]], LF_kind='UV', merge_bins=True)

        [yerr_up, yerr_down] = LyaLF['LF_total_err']
        yerr_up = (yerr_up**2 + LyaLF['poisson_err']**2)**0.5
        yerr_down = (yerr_down**2 + LyaLF['poisson_err']**2)**0.5
        # ax.errorbar(LyaLF['LF_bins'], LyaLF['LF_total'],
        #             yerr=[yerr_down, yerr_up],
        #             fmt='s', ls='', mfc='none', mec='k',
        #             ms=6, ecolor='k', capsize=3,
        #             label='This work')
        ax.errorbar(LyaLF['LF_bins'], LyaLF['LF_boots'],
                    yerr=[yerr_down, yerr_up],
                    fmt='s', ls='', mfc='limegreen', mec='k',
                    ms=6, ecolor='k', capsize=3,
                    label='This work')

        nb_min = nb_list[iii][0]
        nb_max = nb_list[iii][1]

        # this_hiz_nb_mask = (vi_cat_hiz_uv['lya_NB'] >= nb_min) & (vi_cat_hiz_uv['lya_NB'] <= nb_max) & hiz_mask
        # MUV_Arr_hiz = vi_cat_hiz_uv['M_UV'][this_hiz_nb_mask]
        # MUV_e_Arr = vi_cat_hiz_uv['M_UV_err'][this_hiz_nb_mask]
        
        N_bins_UV = 15 + 1
        M_UV_bins = np.linspace(-29, -20, N_bins_UV)

        vol_hiz = 0.
        for field_name in ['W1', 'W2', 'W3']:
            vol_hiz += Lya_effective_volume(nb_min, nb_max, field_name)


        # LF_mat_hiz = []
        # for jjj in range(100):
        #     MUV_Arr = np.random.choice(MUV_Arr_hiz, len(MUV_Arr_hiz), replace=True)

        #     randN = np.random.randn(len(MUV_Arr))
        #     MUV_perturbed = np.empty_like(MUV_Arr)
        #     MUV_perturbed = MUV_Arr + MUV_e_Arr * randN
        #     MUV_perturbed[np.isnan(MUV_perturbed)] = 0.

        #     LF_mat_hiz.append(np.histogram(MUV_perturbed, M_UV_bins,
        #                                    )[0] / (M_UV_bins[1] - M_UV_bins[0]) / vol_hiz)


        # LF_hiz = np.mean(LF_mat_hiz, axis=0)
        # LF_hiz_std = np.std(LF_mat_hiz, axis=0)
        # LF_hiz_err_down = LF_hiz - np.percentile(LF_mat_hiz, axis=0, q=16)
        # LF_hiz_err_up = np.percentile(LF_mat_hiz, axis=0, q=84) - LF_hiz

        # ax.errorbar(bin_centers(M_UV_bins), LF_hiz,
        #             yerr=LF_hiz_std,
        #             fmt='s', ls='', mfc='peru', mec='k',
        #             ms=6, ecolor='k', capsize=3,
        #             label='This work')
    else:
        hiz_mask = vi_cat_hiz['is_hiZ_LAE']
        nb_min = vi_cat_hiz['lya_NB'][hiz_mask].min()
        nb_max = vi_cat_hiz['lya_NB_VI'][hiz_mask].max()

        N_bins_UV = 15 + 1
        M_UV_bins = np.linspace(-29, -20, N_bins_UV)

        nb_min = vi_cat_hiz['lya_NB'][hiz_mask].min()
        nb_max = vi_cat_hiz['lya_NB_VI'][hiz_mask].max()

        vol_hiz = 0.
        for field_name in ['W1', 'W2', 'W3']:
            vol_hiz += Lya_effective_volume(nb_min, nb_max, field_name)

        LF_mat_hiz = []
        for jjj in range(100):
            MUV_Arr = np.random.choice(MUV_Arr_hiz, len(MUV_Arr_hiz), replace=True)

            randN = np.random.randn(len(MUV_Arr))
            MUV_perturbed = np.empty_like(MUV_Arr)
            MUV_perturbed = MUV_Arr + MUV_e_Arr * randN
            MUV_perturbed[np.isnan(MUV_perturbed)] = 0.


            puri_sel = np.interp(MUV_perturbed, puricomp1d_L_bins_c, puri1d)
            comp_sel = np.interp(MUV_perturbed, puricomp1d_L_bins_c, comp1d)
            weights = puri_sel / comp_sel

            LF_mat_hiz.append(np.histogram(MUV_perturbed, M_UV_bins,
                                           weights=weights)[0] / (M_UV_bins[1] - M_UV_bins[0]) / vol_hiz)

        LF_hiz = np.mean(LF_mat_hiz, axis=0)
        LF_hiz_std = np.std(LF_mat_hiz, axis=0)
        LF_hiz_err_down = LF_hiz - np.percentile(LF_mat_hiz, axis=0, q=16)
        LF_hiz_err_up = np.percentile(LF_mat_hiz, axis=0, q=84) - LF_hiz

        ax.errorbar(bin_centers(M_UV_bins), LF_hiz,
                    yerr=LF_hiz_std,
                    fmt='s', ls='', mfc='none', mec='k',
                    ms=6, ecolor='k', capsize=3,
                    label='This work', zorder=99999,)

    # DPL fit
    if iii < 8:
        M_Arr = np.linspace(-31, -18, 1000)
        Phi_fit_16, Phi_fit_50, Phi_fit_84, Phi_fit_05, Phi_fit_95 =\
            load_mcmc_dpl_fit(*nb_list[iii])

        ax.fill_between(M_Arr, Phi_fit_16, Phi_fit_84,
                        alpha=0.2, color='limegreen', lw=0)
        ax.plot(M_Arr, Phi_fit_50,
                color='limegreen', lw=3)

        Phi_fit_16, Phi_fit_50, Phi_fit_84, Phi_fit_05, Phi_fit_95 =\
            load_mcmc_dpl_fit(*nb_list[iii], suffix='_fixed_beta')

        ax.plot(M_Arr, Phi_fit_50,
                color='teal', lw=3, ls='-')
    # else:
    #     M_Arr = np.linspace(-31, -18, 1000)
    #     Phi_fit_16, Phi_fit_50, Phi_fit_84, Phi_fit_05, Phi_fit_95 =\
    #         load_mcmc_dpl_fit(nb_min, nb_max)

    #     ax.fill_between(M_Arr, Phi_fit_16, Phi_fit_84,
    #                     alpha=0.2, color='limegreen', lw=0)
    #     ax.plot(M_Arr, Phi_fit_50,
    #             color='limegreen', lw=2)

    # Liu+22
    if iii == 0:
        ax.errorbar(L22_UV['M_UV'], L22_UV['Phi'],
                    yerr=L22_UV['Phi_err'],
                    fmt='d', mec='aqua', mfc='aqua', ecolor='aqua',
                    capsize=2, label='Liu+22 (z=2.6)', zorder=-80)
    
    # Kulkarni+19 DPL
    for rid in np.atleast_1d(ref_row_idx[iii]):
        this_ref_id = rid
        ref_Phistar = k19_ref_MUV_LF['log10phi'][this_ref_id]
        ref_Mbreak = -k19_ref_MUV_LF['M_'][this_ref_id]
        ref_beta = k19_ref_MUV_LF['alpha'][this_ref_id]
        ref_gamma = k19_ref_MUV_LF['beta'][this_ref_id]
        ref_DPL = double_power_law(M_Arr, ref_Phistar, ref_Mbreak,
                                ref_beta, ref_gamma)
        k19_z = (k19_ref_MUV_LF['zmin'][this_ref_id] + k19_ref_MUV_LF['zmax'][this_ref_id]) * 0.5

        my_distmod = cosmo.distmod(k19_z).value
        ref_distmod = cosmo_ref.distmod(k19_z).value
        my_dV = cosmo.differential_comoving_volume(k19_z).value
        ref_dV = cosmo_ref.differential_comoving_volume(k19_z).value

        ax.plot(M_Arr + my_distmod - ref_distmod, ref_DPL * my_dV / ref_dV,
                label=f'Kulkarni+19 ($z={k19_z:0.2f}$)',
                c=f'C{rid}', ls='--')

    # Pan+21
    if iii == 7:
        ref_DPL = double_power_law(M_Arr, -7.2, -26.7, 1.7, 4)
        ax.plot(M_Arr, ref_DPL,
                label='Pan+21 ($z=3.8$)',
                c='orchid', ls=':')
    if iii == 8:
        ref_DPL = double_power_law(M_Arr, -7.6, -26.6, 1.6, 3.7)
        ax.plot(M_Arr, ref_DPL,
                label='Pan+21 ($z=4.25$)',
                c='orchid', ls=':')

    # Niida+20
    if iii > 7:
        ref_N20 = double_power_law(M_Arr, np.log10(1.01e-7), -25.05, 1.22, 2.9)
        ax.plot(M_Arr, ref_N20, c='teal', label='Niida+20 $(z=5)$', ls='-.')

    if not iii % axs.shape[1]:
        ax.set_ylabel(r'$\Phi$ [Mpc$^{-3}\,\Delta$mag$^{-1}$]', fontsize=20)
    if iii >= len(axs.flatten()) - axs.shape[1]:
        ax.set_xlabel(r'$M_{{\rm UV, 1450}}$ [mag]',
                      fontsize=20)

    
    # Schindler+19
    z_ref = [2.9, 3.25, 3.75, 4.25]
    Phistar_ref = [-6.23, -7.33, -7.65, -8.16]
    Mstar_ref = [-25.58, -27.13, -27.17, -27.57]
    beta_ref = [-1.27, -1.92, -1.7, -1.65]
    gamma_ref = [-3.44, -4.58, -4.52, -4.5]
    if iii == 0:
        jj = 0
        ref_S19 = double_power_law(M_Arr, Phistar_ref[jj], Mstar_ref[jj],
                                   -beta_ref[jj], -gamma_ref[jj])
        ax.plot(M_Arr, ref_S19, c='purple', label=f'Schindler+20 $(z={z_ref[jj]:0.2f})$', ls='-.')
    if iii == 3:
        jj = 1
        ref_S19 = double_power_law(M_Arr, Phistar_ref[jj], Mstar_ref[jj],
                                   -beta_ref[jj], -gamma_ref[jj])
        ax.plot(M_Arr, ref_S19, c='purple', label=f'Schindler+20 $(z={z_ref[jj]:0.2f})$', ls='-.')
    if (iii == 5) or (iii == 6):
        jj = 2
        ref_S19 = double_power_law(M_Arr, Phistar_ref[jj], Mstar_ref[jj],
                                   -beta_ref[jj], -gamma_ref[jj])
        ax.plot(M_Arr, ref_S19, c='purple', label=f'Schindler+20 $(z={z_ref[jj]:0.2f})$', ls='-.')
    if (iii == 7) or (iii == 8):
        jj = 3
        ref_S19 = double_power_law(M_Arr, Phistar_ref[jj], Mstar_ref[jj],
                                   -beta_ref[jj], -gamma_ref[jj])
        ax.plot(M_Arr, ref_S19, c='purple', label=f'Schindler+20 $(z={z_ref[jj]:0.2f})$', ls='-.')



    z_min = (w_central[nb_min] - fwhm_Arr[nb_min] * 0.5) / w_lya - 1
    z_max = (w_central[nb_max] + fwhm_Arr[nb_max] * 0.5) / w_lya - 1
    ax.text(-21.2, 2e-5, fr'\bf z = {z_min:0.2f} - {z_max:0.2f}',
            fontsize=13)


    ax.set_ylim(2e-10, 1e-3)
    ax.set_xlim(-31, -21)
    ax.set_yscale('log')
    ax.invert_xaxis()

    ax.set_yticks(10.**np.arange(-9, -2, 1))

    ax.tick_params(labelsize=17, direction='in', which='both')
    ax.yaxis.set_ticks_position('both')
    ax.xaxis.set_ticks_position('both')

    ax.legend(fontsize=10, loc='upper right')

fig.subplots_adjust(wspace=0.03, hspace=0.05)

fig.savefig('../figures/multiLF_UV.pdf', bbox_inches='tight', pad_inches=0.1,
            facecolor='w')
plt.show()

In [ ]:
from scipy.integrate import trapezoid

c = 299792458 * u.m / u.s

L_units = u.erg * u.s**-1 * u.Hz**-1
Phi_units = u.Mpc**-3 / L_units

ion_em_constant = (1200 / 1450)**0.44 * (912 / 1200)**1.57
f_esc = 1.


def ionizing_emissivity(UV_LF, L_UV_Arr):
    # From E. Giallongo et al. (2015)
    sort_ids = np.argsort(L_UV_Arr)
    x_Arr = L_UV_Arr[sort_ids] * L_units
    y_Arr = (UV_LF / (L_UV_Arr * L_units * (1/2.5) * np.log(10)) * L_UV_Arr)[sort_ids] * Phi_units * L_units**2

    return f_esc * trapezoid(y_Arr, x=x_Arr)

z_list = []
for [nb1, nb2] in nb_list:
    z_min = (w_central[nb1] - fwhm_Arr[nb1] * 0.5) / w_lya - 1
    z_max = (w_central[nb2] + fwhm_Arr[nb2] * 0.5) / w_lya - 1
    z_list.append([z_min, z_max])
z_central_list = [(zmax + zmin) * 0.5 for [zmin, zmax] in z_list]

e_ion_list = []
e_ion_err_list = []

for iii in range(len(nb_list)):
    # DPL fit
    M_Arr = np.linspace(-30, -21, 100)

    app_m = M_Arr + cosmo.distmod(z_central_list[iii]).value - 2.5 * np.log10(1 + z_central_list[iii])
    fnu = 10**(-(app_m + 48.6) / 2.5) * 4 * np.pi * cosmo.luminosity_distance(z_central_list[iii]).to(u.cm).value ** 2
    L_UV_Arr = fnu / (1 + z_central_list[iii])

    Phi_fit_16, Phi_fit_50, Phi_fit_84, Phi_fit_05, Phi_fit_95 =\
        load_mcmc_dpl_fit(*nb_list[iii])


    mask = (Phi_fit_50 > 1e-20)
    e_ion = ionizing_emissivity(Phi_fit_50[mask], L_UV_Arr[mask])
    e_ion_err = (ionizing_emissivity(Phi_fit_84[mask], L_UV_Arr[mask]) - ionizing_emissivity(Phi_fit_16[mask], L_UV_Arr[mask])) * 0.5

    epsilon_units = u.erg * u.s**-1 * u.Mpc**-3 * u.Hz**-1
    print(np.log10(e_ion.to(epsilon_units).value))

    e_ion_list.append(e_ion.to(epsilon_units).value)
    e_ion_err_list.append(e_ion_err.to(epsilon_units).value)

e_ion_list_fb = []
e_ion_err_list_fb = []

for iii in range(len(nb_list)):
    # DPL fit
    M_Arr = np.linspace(-30, -21, 100)

    app_m = M_Arr + cosmo.distmod(z_central_list[iii]).value - 2.5 * np.log10(1 + z_central_list[iii])
    fnu = 10**(-(app_m + 48.6) / 2.5) * 4 * np.pi * cosmo.luminosity_distance(z_central_list[iii]).to(u.cm).value ** 2
    L_UV_Arr = fnu / (1 + z_central_list[iii])

    Phi_fit_16, Phi_fit_50, Phi_fit_84, Phi_fit_05, Phi_fit_95 =\
        load_mcmc_dpl_fit(*nb_list[iii], suffix='_fixed_beta')


    mask = (Phi_fit_50 > 1e-20)
    e_ion = ionizing_emissivity(Phi_fit_50[mask], L_UV_Arr[mask])
    e_ion_err = (ionizing_emissivity(Phi_fit_84[mask], L_UV_Arr[mask]) - ionizing_emissivity(Phi_fit_16[mask], L_UV_Arr[mask])) * 0.5

    epsilon_units = u.erg * u.s**-1 * u.Mpc**-3 * u.Hz**-1
    print(np.log10(e_ion.to(epsilon_units).value))

    e_ion_list_fb.append(e_ion.to(epsilon_units).value)
    e_ion_err_list_fb.append(e_ion_err.to(epsilon_units).value)

In [ ]:
def epsilon_dpl_integrate(Phistar, Mstar, beta, gamma, redshift,
                          M_min=-30, M_max=-21):
    M_xx = np.linspace(M_min, M_max, 1000)
    Phi = double_power_law(M_xx, Phistar, Mstar, -beta, -gamma)

    app_m = M_xx + cosmo.distmod(redshift).value - 2.5 * np.log10(1 + z_central_list[iii])
    fnu = 10**(-(app_m + 48.6) / 2.5) * 4 * np.pi * cosmo.luminosity_distance(redshift).to(u.cm).value ** 2
    L_UV_Arr = fnu / (1 + redshift)

    return ionizing_emissivity(Phi, L_UV_Arr).value

In [ ]:
def eion_HM(z, e0, a, b, c, d):
    return e0 * (1 + z)**a * np.exp(-b * z) / (np.exp(c * z) + d)

fig, ax = plt.subplots(figsize=(6, 5))

ax.errorbar(z_central_list, e_ion_list, yerr=e_ion_err_list,
            ls='', fmt='o', mfc='limegreen', mec='k',
            ecolor='k', capsize=2, ms=8,
            label='This work', zorder=10)

ax.errorbar(np.array(z_central_list) + 0.027, e_ion_list_fb, yerr=e_ion_err_list_fb,
            ls='', fmt='o', mfc='teal', mec='k',
            ecolor='k', capsize=2, ms=8,
            label=r'This work (fixed $\beta=-1.25$)', zorder=9)

MUV_Arr_hiz = [-25.8203611435952, -26.212678406852586, -24.740548987801873, -24.874748293431768, -25.174493132247367, -25.602635160990275, -23.89222607237893, -23.9203479579364, -24.690615455415514, -24.7771626754886, -24.15656403384465, -25.09473542804817, -24.660957929355966, -27.48978954114158, -25.63107733857017, -27.11613266030886, -25.009457289541547, -24.879800171402128]
MUV_e_Arr = [0.11818163122030359, 0.029330895423108582, 0.21122928347527536, 0.18454288586918452, 0.16338634354078094, 0.04101937457377991, 0.13203216347706037, 0.12600382326959442, 0.17985356707774844, 0.22956735374987605, 0.4327381891239168, 0.25491923639678593, 0.1842595221993075, 0.040392473665365, 0.10596378281378038, 0.0313305870083058, 0.17442288222306845, 0.21355673942153533]
redshift_list = np.array([5.21221219574391, 4.474347479167866, 4.638865810622948, 4.805851917049857, 4.305716189426406, 5.050161639260654, 4.474347479167866, 4.719479793035939, 4.885643307805572, 5.050161639260654, 4.305716189426406, 4.394556088412151, 4.474347479167866, 4.474347479167866, 4.474347479167866, 4.474347479167866, 4.719479793035939, 4.885643307805572])

app_m = np.array(MUV_Arr_hiz) + cosmo.distmod(redshift_list).value - 2.5 * np.log10(1 + redshift_list)
fnu = 10**(-(app_m + 48.6) / 2.5) * 4 * np.pi * cosmo.luminosity_distance(redshift_list).to(u.cm).value ** 2
L_UV_Arr = fnu / (1 + redshift_list)
eion_hiz = np.sum(L_UV_Arr) / vol_hiz

app_m = np.array(MUV_Arr_hiz) + np.array(MUV_e_Arr) + cosmo.distmod(redshift_list).value - 2.5 * np.log10(1 + redshift_list)
fnu = 10**(-(app_m + 48.6) / 2.5) * 4 * np.pi * cosmo.luminosity_distance(redshift_list).to(u.cm).value ** 2
L_UV_Arr = fnu / (1 + redshift_list)
eion_hiz_err = eion_hiz - np.sum(L_UV_Arr) / vol_hiz

ax.errorbar(4.75, eion_hiz, yerr=eion_hiz_err * 10,
            ls='', fmt='o', mfc='limegreen', mec='k', mew=1,
            ecolor='k', capsize=2, ms=8,
            lolims=True, zorder=9)

### Fit to a simple power-law
# def linear_fit(x, a, b):
#     return b + a * np.array(x)
# popt, pcov = curve_fit(linear_fit, z_central_list, np.log10(e_ion_list))
# xx = np.linspace(2.5, 5, 100)
# ax.plot(xx, 10**linear_fit(xx, popt[0], popt[1]),
#         ls='--', lw=2.5, c='darkgreen', zorder=8,
#         label='This work (Power-law fit)')
# print(popt)
# print(pcov)

## REFs
z_xx = np.linspace(0, 10, 1000)
ax.plot(z_xx,
        eion_HM(z_xx, pow(10, 23.91), 8.26, 1.3, 1.62, 13.6),
        ls='-.', lw=1.5, label='Kulkarni+19', c='darkgoldenrod', zorder=-90)


# Pan22 params
z_ref = [3.8, 4.25, 4.7]
Phistar_ref = [-7.2, -7.6, -7.7]
Phistar_err_ref =[0.3, 0.5, 0.7]
Mstar_ref = [-26.7, -26.6, -26.3]
Mstar_err_ref = [0.4, 0.9, 1.5]
beta_ref = [-1.7, -1.6, -1.7]
beta_err_ref = [0.2, 0.5, 0.5]
gamma_ref = [-4, -3.7, -3.2]
gamma_err_ref = [0.4, 0.6, 0.5]
eion_ref = []
for z, Phistar, Mstar, beta, gamma in zip(z_ref, Phistar_ref, Mstar_ref, beta_ref, gamma_ref):
    eion_ref.append(epsilon_dpl_integrate(Phistar, Mstar, beta, gamma, z))
ax.errorbar(z_ref, eion_ref, xerr=0.25,
            fmt='^', color='violet', ms=7, label='Pan+22')

# Schindler+19
z_ref = [2.9, 3.25, 3.75, 4.25]
Phistar_ref = [-6.23, -7.33, -7.65, -8.16]
Phistar_err_ref = [0, 0, 0, 0]
Mstar_ref = [-25.58, -27.13, -27.17, -27.57]
Mstar_err_ref = [0.22, 0.21, 0.28, 0.24]
beta_ref = [-1.27, -1.92, -1.7, -1.65]
beta_err_ref = [0.2, 0.16, 0.66, 0.46]
gamma_ref = [-3.44, -4.58, -4.52, -4.5]
gamma_err_ref = [0.07, 0.18, 0.15, 0.18]
eion_ref = []
for z, Phistar, Mstar, beta, gamma in zip(z_ref, Phistar_ref, Mstar_ref, beta_ref, gamma_ref):
    eion_ref.append(epsilon_dpl_integrate(Phistar, Mstar, beta, gamma, z))
ax.errorbar(z_ref, eion_ref, xerr=0.25,
            fmt='s', color='crimson', label='Schindler+19', ms=7)

# Liu+22
this_L22 = pd.read_csv('/data/alberto/PAUS_data/literature_UV_LFs/Liu_M_UV_z2.1.csv')
this_z = 2.1
app_m = this_L22['M_UV'].to_numpy() + cosmo.distmod(this_z).value - 2.5 * np.log10(1 + this_z)
fnu = 10**(-(app_m + 48.6) / 2.5) * 4 * np.pi * cosmo.luminosity_distance(this_z).to(u.cm).value ** 2
this_LUV = fnu / (1 + this_z)
this_eion = ionizing_emissivity(this_L22['Phi'], this_LUV).value
ax.errorbar(this_z, this_eion,
            fmt='D', color='tan', ms=7, label='Liu+22')

this_L22 = pd.read_csv('/data/alberto/PAUS_data/literature_UV_LFs/Liu_M_UV_z2.6.csv')
this_z = 2.6
app_m = this_L22['M_UV'].to_numpy() + cosmo.distmod(this_z).value - 2.5 * np.log10(1 + this_z)
fnu = 10**(-(app_m + 48.6) / 2.5) * 4 * np.pi * cosmo.luminosity_distance(this_z).to(u.cm).value ** 2
this_LUV = fnu / (1 + this_z)
this_eion = ionizing_emissivity(this_L22['Phi'], this_LUV).value
ax.errorbar(this_z, this_eion,
            fmt='D', color='tan', ms=7)


# Matsuoka+18
eion_ref = epsilon_dpl_integrate(-8.1, -24.9, -1.23, -2.73, 6.1)
ax.errorbar(6.1, eion_ref, xerr=0.4,
            fmt='o', zorder=-99, c='blue', label='Matsuoka+18', ms=7)


# Niida+20
eion_ref = epsilon_dpl_integrate(np.log10(1.01e-7), -25.05, -1.22, -2.9, 5)
ax.errorbar(4.9, eion_ref, xerr=0.35,
            fmt='d', zorder=-99, c='indigo', label='Niida+20', ms=7)


# Akiyama+18
eion_ref = epsilon_dpl_integrate(np.log10(2.66e-7), -25.36, -1.3, -3.11, 4)
ax.errorbar(4, eion_ref, xerr=0.3,
            fmt='*', zorder=-99, c='darkviolet', label='Akiyama+18', ms=10)
            

handles, labels = ax.get_legend_handles_labels()
order = np.arange(len(handles))
order[:3] = [1, 2, 0]
ax.legend([handles[idx] for idx in order],[labels[idx] for idx in order],
          fontsize=12, ncol=2, loc='upper left', frameon=True)

ax.set_xlabel('Redshift')
ax.set_ylabel(r'$\epsilon_{1450}$ [erg\,s$^{-1}$\,Hz$^{-1}$\,Mpc$^{-3}$]')

ax.set_yscale('log')
ax.set_ylim(3e22, 1e27)
ax.set_xlim(2, 7)

ax.tick_params(direction='in', which='both')
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')

fig.savefig('../figures/eion_1450.pdf', bbox_inches='tight', pad_inches=0.1,
            facecolor='w')
plt.show()

In [ ]:
# survey_list = ['W3', 'W1', 'W2']
# nb_list = [[0, 2], [2, 4], [4, 6], [6, 8],
#            [8, 10], [10, 12], [12, 14], [14, 16], [16, 18]]
# nb_list = [[0, 3], [2, 5], [4, 7], [6, 9], [8, 11], [10, 13], [12, 15], [14, 18]]

# fig, axs = plt.subplots(3, 3, figsize=(14, 14), sharey=True, sharex=True,
#                         width_ratios=[1, 1, 1], height_ratios=[1, 1, 1])

# reference_LFs = [[refLF.tt23_z2630, refLF.s18d],
#                  [refLF.tt23_z2833, refLF.u08, refLF.s18b],
#                  [refLF.tt23_z2833, refLF.u08, refLF.s18c, refLF.ds20_z324,
#                   refLF.l22_z324],
#                  [refLF.tt23_z3135, refLF.s18e, refLF.ds20_z324, refLF.l22_z324],
#                  [refLF.tt23_z3338, refLF.s18f],
#                  [],
#                  [],
#                  [],
#                  []]

# for iii, ax in enumerate(axs.flatten()):
#     if len(nb_list[iii]) == 0:
#         continue

#     for reg in survey_list:
#         LyaLF = load_combined_LF([reg], [nb_list[iii]])

#         [yerr_up, yerr_down] = LyaLF['LF_total_err']
#         mask = (yerr_down == 0) & (LyaLF['LF_total'] > 0) # Where yerr_down boots is zero
#         yerr_down[mask] = LyaLF['LF_total'][mask]
#         mask = (yerr_up < LyaLF['poisson_err']) & (LyaLF['LF_total'] > 0)
#         yerr_up[mask] = LyaLF['poisson_err'][mask]
#         mask = (yerr_down < LyaLF['poisson_err']) & (LyaLF['LF_total'] > 0)
#         yerr_down[mask] = LyaLF['poisson_err'][mask]
#         ax.errorbar(LyaLF['LF_bins'], LyaLF['LF_total'],
#                     yerr=[yerr_down, yerr_up],
#                     fmt='s', ls='', mec='k',
#                     ms=6, ecolor='k', capsize=3,
#                     label=reg)

#     LyaLF = load_combined_LF(survey_list, [nb_list[iii]])

#     [yerr_up, yerr_down] = LyaLF['LF_total_err']
#     mask = (yerr_down == 0) & (LyaLF['LF_total'] > 0) # Where yerr_down boots is zero
#     yerr_down[mask] = LyaLF['LF_total'][mask]
#     mask = (yerr_up < LyaLF['poisson_err']) & (LyaLF['LF_total'] > 0)
#     yerr_up[mask] = LyaLF['poisson_err'][mask]
#     mask = (yerr_down < LyaLF['poisson_err']) & (LyaLF['LF_total'] > 0)
#     yerr_down[mask] = LyaLF['poisson_err'][mask]
#     ax.errorbar(LyaLF['LF_bins'], LyaLF['LF_total'],
#                 yerr=[yerr_down, yerr_up],
#                 fmt='s', ls='', mec='k',
#                 ms=6, ecolor='k', capsize=3,
#                 label='All')

#     if not iii % axs.shape[1]:
#         ax.set_ylabel(r'$\Phi$ [Mpc$^{-3}\,\Delta\log_{10}L^{-1}$]', fontsize=20)
#     if iii >= len(axs.flatten()) - axs.shape[1]:
#         ax.set_xlabel(r'$\log_{10}(L_{\mathrm{Ly}\alpha}/\mathrm{erg\,s}^{-1})$',
#                       fontsize=20)


#     nb_min = nb_list[iii][0]
#     nb_max = nb_list[iii][1]
#     z_min = (w_central[nb_min] - fwhm_Arr[nb_min] * 0.5) / w_lya - 1
#     z_max = (w_central[nb_max] + fwhm_Arr[nb_max] * 0.5) / w_lya - 1
#     ax.text(42.8, 1e-5, fr'\bf z = {z_min:0.2f} - {z_max:0.2f}',
#             fontsize=13)


#     ax.set_ylim(5e-8, 2e-5)
#     ax.set_xlim(42.7, 45.8)
#     ax.set_yscale('log')

#     ax.tick_params(labelsize=17, direction='in', which='both')
#     ax.set_yticks(np.logspace(-7, -5, 3))
#     xticks = np.arange(43, 46, 0.5)
#     ax.set_xticks(xticks)
#     # ax.set_xticklabels(f'{tick:0.1f}' for tick in xticks)
#     ax.yaxis.set_ticks_position('both')
#     ax.xaxis.set_ticks_position('both')
#     # ax.set_xticks(np.arange(42.5, 46, 0.5))

#     ax.legend(fontsize=10)

# fig.subplots_adjust(wspace=0.03, hspace=0.05)

# fig.savefig('../figures/multiLF_individual_NBs.pdf', bbox_inches='tight', pad_inches=0.1,
#             facecolor='w')
# plt.show()

In [ ]:
pathname = '/home/alberto/cosmos/PAUS_LAEs/curve_fit/schechter_fit_parameters.csv'
sch_params = pd.read_csv(pathname)

pathname = '/home/alberto/cosmos/PAUS_LAEs/curve_fit/schechter_fit_parameters_fixed_Phistar.csv'
sch_params_fP = pd.read_csv(pathname)

pathname = '/home/alberto/cosmos/PAUS_LAEs/curve_fit/schechter_fit_parameters_fixed_Lstar.csv'
sch_params_fL = pd.read_csv(pathname)

pathname = '/home/alberto/cosmos/PAUS_LAEs/curve_fit/schechter_fit_parameters_fixed_alpha.csv'
sch_params_fa = pd.read_csv(pathname)

display(sch_params)

fig, [axes_leg, axes] = plt.subplots(2, 3, figsize=(12, 3.6), height_ratios=[0.1, 5])

axleg = axes_leg[1]

nb_min_Arr = np.array(sch_params['nb_min']).astype(int)
nb_max_Arr = np.array(sch_params['nb_max']).astype(int)
nb_max_Arr[nb_max_Arr > 39] = 39
z_min_Arr = (w_central[nb_min_Arr] - fwhm_Arr[nb_min_Arr] * 0.5) / w_lya - 1
z_max_Arr = (w_central[nb_max_Arr] + fwhm_Arr[nb_max_Arr] * 0.5) / w_lya - 1
z_mid_Arr = (z_min_Arr + z_max_Arr) * 0.5
# z_mid_Arr[-1] = 4.75

mask = (np.arange(len(sch_params)) != 8)
fit_param_list = ['Phistar', 'Lstar', 'alpha']
fit_param_label_list = [r'$\log_{10}(\Phi^* / {\rm Mpc}^{-3})$',
                        r'$\log_{10}(L^* / {\rm erg\,s}^{-1})$',
                        r'$\alpha$']

yticks_list = [np.arange(-9, -5, 0.5),
               np.arange(44, 48, 0.5),
               np.arange(-3, 1, 0.5)]

ylims_list = [(-8.3, -6.),
              (44, 46.5),
              (-3, 0)]

for i, ax in enumerate(axes.flatten()):
    this_param = fit_param_list[i]
    ax.errorbar(z_mid_Arr[mask], sch_params[this_param][mask],
                yerr=[sch_params[f'{this_param}_err_down'][mask],
                    sch_params[f'{this_param}_err_up'][mask]],
                ls='', fmt='s', mfc='limegreen', mec='k',
                ecolor='limegreen', capsize=2, ms=7,
                )

    if this_param != 'alpha':
        ax.errorbar(z_mid_Arr[mask] + 0.04, sch_params_fa[this_param][mask],
                    yerr=[sch_params_fa[f'{this_param}_err_down'][mask],
                        sch_params_fa[f'{this_param}_err_up'][mask]],
                    ls='', fmt='o', mfc='crimson', mec='k',
                    ecolor='crimson', capsize=2, ms=7,
                    )
    if this_param != 'Lstar':
        ax.errorbar(z_mid_Arr[mask] + 0.08, sch_params_fL[this_param][mask],
                    yerr=[sch_params_fL[f'{this_param}_err_down'][mask],
                        sch_params_fL[f'{this_param}_err_up'][mask]],
                    ls='', fmt='^', mfc='teal', mec='k',
                    ecolor='teal', capsize=2, ms=7,
                    )

    this_mean_param = np.average(sch_params[this_param][:-2],
                              weights=sch_params[f'{this_param}_err_up'][:-2]**-2)
    print(this_mean_param)
    print()

    ax.axhline(this_mean_param, lw=2.5, ls='--', c='limegreen')


    ax.set_ylabel(fit_param_label_list[i])
    ax.set_xlabel('Redshift')

    ax.set_xticks(np.arange(2.5, 6, 0.5))
    ax.set_yticks(yticks_list[i])

    ax.tick_params(labelsize=17, direction='in', which='both')
    ax.yaxis.set_ticks_position('both')
    ax.xaxis.set_ticks_position('both')

    ax.set_ylim(*ylims_list[i])
    ax.set_xlim(2.4, 4.5)


axleg.errorbar([], [],
            ls='', fmt='s', mfc='limegreen', mec='k',
            ecolor='limegreen', capsize=2, ms=7,
            label='Free parameters')
axleg.errorbar([], [],
            ls='', fmt='o', mfc='crimson', mec='k',
            ecolor='crimson', capsize=2, ms=7,
            label=r'Fixed $\alpha=-1.69$')
axleg.errorbar([], [],
            ls='', fmt='^', mfc='teal', mec='k',
            ecolor='teal', capsize=2, ms=7,
            label=r'Fixed $\log_{10}(L^* / {\rm erg\,s}^{-1})=45$')
axleg.legend(ncol=3, loc='center', bbox_to_anchor=[0.5, 0.5], frameon=False)

for axi in axes_leg:
    axi.axis('off')


fig.subplots_adjust(wspace=0.35)

fig.savefig('../figures/Schechter_parameters.pdf', bbox_inches='tight', pad_inches=0.1,
            facecolor='w')
plt.show()

# fig, ax = plt.subplots(figsize=(6, 4))

# ax.scatter(z_mid_Arr[np.arange(8)], chi2)
# ax.scatter(z_mid_Arr[np.arange(8)], chi2_fa)
# ax.scatter(z_mid_Arr[np.arange(8)], chi2_fl)

# ax.set_xlabel(r'$z$')
# ax.set_ylabel(r'$\log_{10}\chi^2$')

# plt.show()

In [ ]:
from uncertainties import ufloat

pathname = '/home/alberto/cosmos/PAUS_LAEs/curve_fit/UV_dpl_fit_parameters_fixed_M.csv'
dpl_params_fM = pd.read_csv(pathname)

pathname = '/home/alberto/cosmos/PAUS_LAEs/curve_fit/UV_dpl_fit_parameters_fixed_gamma.csv'
dpl_params_fg = pd.read_csv(pathname)

pathname = '/home/alberto/cosmos/PAUS_LAEs/curve_fit/UV_dpl_fit_parameters_fixed_beta.csv'
dpl_params_fb = pd.read_csv(pathname)

pathname = '/home/alberto/cosmos/PAUS_LAEs/curve_fit/UV_dpl_fit_parameters.csv'
dpl_params = pd.read_csv(pathname)
display(dpl_params)


fig, axes = plt.subplots(2, 2, figsize=(7, 7), sharex=True)

nb_min_Arr = np.array(dpl_params['nb_min']).astype(int)
nb_max_Arr = np.array(dpl_params['nb_max']).astype(int)
nb_max_Arr[nb_max_Arr > 39] = 39
z_min_Arr = (w_central[nb_min_Arr] - fwhm_Arr[nb_min_Arr] * 0.5) / w_lya - 1
z_max_Arr = (w_central[nb_max_Arr] + fwhm_Arr[nb_max_Arr] * 0.5) / w_lya - 1
z_mid_Arr = (z_min_Arr + z_max_Arr) * 0.5
# z_mid_Arr[-1] = 4.75

mask = (np.arange(len(dpl_params)) != 8)
fit_param_list = ['Phistar', 'Mbreak', 'beta', 'gamma']
fit_param_label_list = [r'$\log_{10}(\Phi^* / {\rm Mpc}^{-3})$',
                        r'$M^*$',
                        r'$\beta$ (faint-end)',
                        r'$\gamma$ (bright-end)']

k19_params_list = ['log10phi', 'M_', 'beta', 'alpha']
k19_eup_list = ['E_log10phi', 'E_M_', 'E_beta', 'E_alpha']
k19_edown_list = ['e_log10phi_lc', 'e_m__lc', 'e_beta_lc', 'e_alpha_lc']

# yticks_list = [np.arange(-9, -5, 0.5),
#                np.arange(44, 48, 0.5),
#                np.arange(-3, 0, 0.5)]

ylims_list = [(-9.5, -5.),
              (-30, -24),
              (-3, -0.6),
              (-8, -1)]

for i, ax in enumerate(axes.flatten()):
    this_param = fit_param_list[i]
    if i < 2:
        this_my_params = dpl_params[this_param][mask]
    else:
        this_my_params = -dpl_params[this_param][mask]

    ax.errorbar(z_mid_Arr[mask], this_my_params,
                yerr=[dpl_params[f'{this_param}_err_down'][mask],
                    dpl_params[f'{this_param}_err_up'][mask]],
                ls='', fmt='s', mfc='limegreen', mec='k',
                ecolor='limegreen', capsize=2, ms=7,
                label='This work')


    this_param = fit_param_list[i]
    if i < 2:
        this_my_params = dpl_params_fb[this_param][mask]
    else:
        this_my_params = -dpl_params_fb[this_param][mask]

    if this_param != 'beta':
        ax.errorbar(z_mid_Arr[mask] + 0.04, this_my_params,
                    yerr=[dpl_params_fb[f'{this_param}_err_down'][mask],
                        dpl_params_fb[f'{this_param}_err_up'][mask]],
                    ls='', fmt='o', mfc='teal', mec='k',
                    ecolor='teal', capsize=2, ms=7,
                    label=r'This work (fixed $\beta=-1.25$)'
                    )

    # this_param = fit_param_list[i]
    # if i < 2:
    #     this_my_params = dpl_params_fM[this_param][mask]
    # else:
    #     this_my_params = -dpl_params_fM[this_param][mask]

    # if this_param != 'goamma':
    #     ax.errorbar(z_mid_Arr[mask] + 0.04, this_my_params,
    #                 yerr=[dpl_params_fM[f'{this_param}_err_down'][mask],
    #                     dpl_params_fM[f'{this_param}_err_up'][mask]],
    #                 ls='', fmt='^', mfc='crimson', mec='k',
    #                 ecolor='crimson', capsize=2, ms=7,
    #                 )

    if i == 0:
        this_k19_params = k19_ref_MUV_LF[k19_params_list[i]]
    else:
        this_k19_params = -k19_ref_MUV_LF[k19_params_list[i]]

    ax.errorbar(k19_ref_MUV_LF['z'], this_k19_params,
                yerr=[k19_ref_MUV_LF[k19_eup_list[i]], k19_ref_MUV_LF[k19_edown_list[i]]],
                fmt='D', color='orange', zorder=-99, label='Kulkarni+19',
                alpha=0.7)

    # Pan22 params
    z_ref = [3.8, 4.25, 4.7]
    Phistar_ref = [-7.2, -7.6, -7.7]
    Phistar_err_ref =[0.3, 0.5, 0.7]
    Mstar_ref = [-26.7, -26.6, -26.3]
    Mstar_err_ref = [0.4, 0.9, 1.5]
    beta_ref = [-1.7, -1.6, -1.7]
    beta_err_ref = [0.2, 0.5, 0.5]
    gamma_ref = [-4, -3.7, -3.2]
    gamma_err_ref = [0.4, 0.6, 0.5]

    if i == 0:
        ax.errorbar(z_ref, Phistar_ref, yerr=Phistar_err_ref,
                    fmt='^', zorder=-99, c='violet', label='Pan+21', alpha=0.7)
    if i == 1:
        ax.errorbar(z_ref, Mstar_ref, yerr=Mstar_err_ref,
                    fmt='^', zorder=-99, c='violet', alpha=0.7)
    if i == 2:
        ax.errorbar(z_ref, beta_ref, yerr=beta_err_ref,
                    fmt='^', zorder=-99, c='violet', alpha=0.7)
    if i == 3:
        ax.errorbar(z_ref, gamma_ref, yerr=gamma_err_ref,
                    fmt='^', zorder=-99, c='violet', alpha=0.7)

    # Schindler+19
    z_ref = [2.9, 3.25, 3.75, 4.25]
    Phistar_ref = [-6.23, -7.33, -7.65, -8.16]
    Phistar_err_ref = [0, 0, 0, 0]
    Mstar_ref = [-25.58, -27.13, -27.17, -27.57]
    Mstar_err_ref = [0.22, 0.21, 0.28, 0.24]
    beta_ref = [-1.27, -1.92, -1.7, -1.65]
    beta_err_ref = [0.2, 0.16, 0.66, 0.46]
    gamma_ref = [-3.44, -4.58, -4.52, -4.5]
    gamma_err_ref = [0.07, 0.18, 0.15, 0.18]

    if i == 0:
        ax.errorbar(z_ref, Phistar_ref, yerr=Phistar_err_ref,
                    fmt='s', zorder=-99, c='crimson', label='Schindler+19', alpha=0.7)
    if i == 1:
        ax.errorbar(z_ref, Mstar_ref, yerr=Mstar_err_ref,
                    fmt='s', zorder=-99, c='crimson', alpha=0.7)
    if i == 2:
        ax.errorbar(z_ref, beta_ref, yerr=beta_err_ref,
                    fmt='s', zorder=-99, c='crimson', alpha=0.7)
    if i == 3:
        ax.errorbar(z_ref, gamma_ref, yerr=gamma_err_ref,
                    fmt='s', zorder=-99, c='crimson', alpha=0.7)

    # Matsuoka+18
    if i == 0:
        ax.errorbar([], [], yerr=[],
                    fmt='o', zorder=-99, c='blue', label='Matsuoka+18', alpha=0.7)
    if i == 1:
        ax.errorbar(6.1, -24.9, yerr=np.array([[0.75, 0.9]]).T,
                    fmt='o', zorder=-99, c='blue', alpha=0.7)
    if i == 2:
        ax.errorbar(6.1, -1.23, yerr=np.array([[0.44, 0.34]]).T,
                    fmt='o', zorder=-99, c='blue', alpha=0.7)
    if i == 3:
        ax.errorbar(6.1, -2.73, yerr=np.array([[0.23, 0.31]]).T,
                    fmt='o', zorder=-99, c='blue', alpha=0.7)

    # Niida+20
    if i == 0:
        ax.errorbar(5, np.log10(1.01e-7), yerr=0.21/1.01,
                    fmt='d', zorder=-99, c='indigo', label='Niida+20', alpha=0.7)
    if i == 1:
        ax.errorbar(5, -25.05, yerr=np.array([[0.1, 0.24]]).T,
                    fmt='d', zorder=-99, c='indigo', alpha=0.7)
    if i == 2:
        ax.errorbar(5, -1.22, yerr=np.array([[0.03, 0.10]]).T,
                    fmt='d', zorder=-99, c='indigo', alpha=0.7)
    if i == 3:
        ax.errorbar(5, -2.9,
                    fmt='d', zorder=-99, c='indigo', alpha=0.7)

    # Akiyama+18
    if i == 0:
        ax.errorbar(4, np.log10(2.66e-7), yerr=0.05/2.66,
                    fmt='*', zorder=-99, c='darkviolet', label='Akiyama+18', ms=10, alpha=0.7)
    if i == 1:
        ax.errorbar(4, -25.36, yerr=0.13,
                    fmt='*', zorder=-99, c='darkviolet', ms=10, alpha=0.7)
    if i == 2:
        ax.errorbar(4, -1.3, yerr=0.05,
                    fmt='*', zorder=-99, c='darkviolet', ms=10, alpha=0.7)
    if i == 3:
        ax.errorbar(4, -3.11, yerr=0.07,
                    fmt='*', zorder=-99, c='darkviolet', ms=10, alpha=0.7)

    ########

    if i == 0:
        ax.legend(fontsize=13,
                  loc=(-0.2, 1.05), ncol=3, frameon=False)

    if i == 2:
        ax.axhline(-1.25, ls='--', lw=2, color='teal', zorder=0)

    if i % 2:
        ax.yaxis.tick_right()
        ax.yaxis.set_label_position('right')

    ax.set_ylabel(fit_param_label_list[i])

    if i > 1:
        ax.set_xlabel('Redshift')

    ax.set_xticks(np.arange(0, 8, 1))

    ax.tick_params(labelsize=17, direction='in', which='both')
    ax.yaxis.set_ticks_position('both')
    ax.xaxis.set_ticks_position('both')

    ax.set_xlim(0.9, 6.5)
    ax.set_ylim(*ylims_list[i])


fig.subplots_adjust(hspace=0.05, wspace=0.1)

fig.savefig('../figures/DPL_parameters.pdf', bbox_inches='tight', pad_inches=0.1,
            facecolor='w')
plt.show()

In [ ]:
survey_list = ['W3', 'W1', 'W2']

fig, ax = plt.subplots(figsize=(6, 5))

LFs_mult_list = []
LFs_err_mult_list = []

for iii in range(14, 18 + 1):
    LyaLF = load_combined_LF(survey_list, [[iii, iii]])
    LFs_mult_list.append(LyaLF['LF_total'])
    LFs_err_mult_list.append(LyaLF['LF_total_err'])

    ax.errorbar(LyaLF['LF_bins'] + iii * 0.01, LyaLF['LF_total'],
                yerr=LyaLF['LF_total_err'],
                fmt='s', ls='', mec='k',
                ms=6, ecolor='k', capsize=3,
                label=f'z = {z_NB(iii):0.2f}')


ax.set_ylabel(r'$\Phi$ [Mpc$^{-3}\,\Delta\log_{10}L^{-1}$]', fontsize=20)
ax.set_xlabel(r'$\log_{10}(L_{\mathrm{Ly}\alpha}/\mathrm{erg\,s}^{-1})$',
                fontsize=20)

ax.set_ylim(5e-8, 2e-5)
ax.set_xlim(42.7, 45.7)
ax.set_yscale('log')

ax.legend()

ax.tick_params(labelsize=17, direction='in', which='both')
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')

ax.legend(fontsize=10)

plt.show()

In [ ]:
# Lmin, Lmax = 43.6, 47
# # Lmin, Lmax = 44, 47
# LF_bins = LyaLF['LF_bins']
# bin_width = LF_bins[1] - LF_bins[0]
# mask = (LF_bins < Lmax) & (LF_bins > Lmin)
# mask_Lx = (np.log10(Lx) < Lmax) & (np.log10(Lx) > Lmin)

# LF_int_list = []
# LF_int_err_up_list = []
# LF_int_err_down_list = []
# for nb in range(19):
#     Lx = np.logspace(43.5, 46, 500)
#     Phi_fit_16, Phi_fit_50, Phi_fit_84, Phi_fit_05, Phi_fit_95 =\
#         load_mcmc_schechter_fit(nb, nb)
#     LF_int_list.append(trapezoid(Phi_fit_50, np.log10(Lx)))
#     LF_int_err_up_list.append((trapezoid(Phi_fit_84, np.log10(Lx))
#                             - trapezoid(Phi_fit_50, np.log10(Lx))))
#     LF_int_err_down_list.append((trapezoid(Phi_fit_50, np.log10(Lx))
#                             - trapezoid(Phi_fit_16, np.log10(Lx))))

# z_list = []
# this_nb_list = [[ii, ii] for ii in range(19)]
# for [nb1, nb2] in this_nb_list:
#     z_min = (w_central[nb1] - fwhm_Arr[nb1] * 0.5) / w_lya - 1
#     z_max = (w_central[nb2] + fwhm_Arr[nb2] * 0.5) / w_lya - 1
#     z_list.append([z_min, z_max])

# z_central_list = [(zmax + zmin) * 0.5 for [zmin, zmax] in z_list]

# # Load SDSS Xmatch
# sdss_xm = pd.read_csv('/home/alberto/almacen/PAUS_data/catalogs/Xmatch_SDSS_W3.csv')

# fig, ax = plt.subplots(figsize=(6, 4))

# # ax.axhline(pl_int * 1e6, ls='--', c='r', lw=3,
# #            label='LF power-law fit')
# ax.errorbar(z_central_list, np.array(LF_int_list) * 1e6,
#             yerr=np.array([LF_int_err_down_list, LF_int_err_up_list]) * 1e6,
#             xerr=[(zmax - zmin) * 0.5 for [zmin, zmax] in z_list],
#             fmt='o', ms=8, capsize=2,
#             ecolor='k', mfc='limegreen', mec='k',
#             label=r'This work', zorder=99)


# # Fit to a power-law
# def powerlaw_fit(x, a, b):
#     return a * np.power(x, b)
# popt, pcov = curve_fit(powerlaw_fit, z_central_list, LF_int_list,
#                     #    sigma=LF_int_err_list,
#                        )
# ax.plot(z_central_list, powerlaw_fit(z_central_list, *popt) * 1e6,
#         ls='--', lw=2.5, c='darkgreen', zorder=95,
#         label='Power-law fit')
# print(f'{popt=}')
# print(f'{pcov=}')


# # Load miniJPAS data
# z_central_list = np.load('/home/alberto/cosmos/LAEs/npy/z_central_list.npy')
# LF_int = np.load('/home/alberto/cosmos/LAEs/npy/LF_int_list.npy')
# LF_int_yerr = np.load('/home/alberto/cosmos/LAEs/npy/LF_int_yerr_list.npy')
# LF_int_xerr = np.load('/home/alberto/cosmos/LAEs/npy/LF_int_xerr_list.npy')
# ax.errorbar(z_central_list, LF_int * 1e6,
#             yerr=LF_int_yerr * 1e6,
#             xerr=LF_int_xerr,
#             fmt='s', ms=7, capsize=2,
#             ecolor='gray', mfc='r', mec='gray',
#             label=r'miniJPAS\&J-NEP', zorder=-1)


# # Plot SDSS QSOs
# z_bins = np.linspace(1.8, 4.3, 15)
# z_bin_c = bin_centers(z_bins)
# vol_Arr = [z_volume(z_bins[i], z_bins[i + 1], 14.76) for i in range(len(z_bins) - 1)]
# h, _ = np.histogram(sdss_xm['z_best'][sdss_xm['L_lya'] > 43.6], z_bins)
# ax.plot(z_bin_c, h / vol_Arr * 1e6,
#         c='seagreen', label='SDSS QSO LAEs', zorder=-2)

# # PD16
# # area_obs = 14.76
# # model = pd.read_csv('/home/alberto/cosmos/LAEs/MyMocks/csv/PD2016-QSO_LF.csv')
# # counts_model_2D = model.to_numpy()[:-1, 1:-1].astype(float) * 1e-4 * area_obs
# # z_xx = np.arange(0, 6.5, 1)
# # vol_Arr = [z_volume(z_xx[i], z_xx[i + 1], 14.76) for i in range(len(z_xx) - 1)]
# # z_model = counts_model_2D.sum(axis=0) / vol_Arr
# # ax.plot(bin_centers(z_xx), z_model * 1e6,
# #         marker='d', mfc='orange', mec='k', ms=12,
# #         ls='', zorder=96)


# handles, labels = ax.get_legend_handles_labels()
# order = np.array([2, 0, 3, 1])
# ax.legend(np.array(handles, dtype=object)[order],
#             np.array(labels, dtype=object)[order],
#             fontsize=12, ncol=1, loc=0)

# ax.tick_params(labelsize=18, direction='in', which='both')
# ax.yaxis.set_ticks_position('both')
# ax.xaxis.set_ticks_position('both')

# ax.set(ylim=(0, 7), xlim=(1.9, 4.3))
# ax.set_ylabel(r'Integrated $\Phi$ [10$^{-6}$ Mpc$^{-3}$]', fontsize=18)
# ax.set_xlabel('Redshift', fontsize=18)

# fig.savefig('../figures/Integrated_LF.pdf', pad_inches=0.1, bbox_inches='tight',
#             facecolor='w')
# plt.show()

In [ ]:
from scipy.integrate import trapezoid

L_min, L_max = 43.5, 47
LF_bins = LyaLF['LF_bins']

nb_min = vi_cat_hiz['lya_NB'][hiz_mask].min()
nb_max = vi_cat_hiz['lya_NB_VI'][hiz_mask].max()
nb_fit_list = nb_list + [[nb_min, nb_max]]
# nb_fit_list = [[jj, jj] for jj in range(19)]

LF_int_list = []
LF_int_err_up_list = []
LF_int_err_down_list = []
for nb1, nb2 in nb_fit_list:
    Lx = np.logspace(L_min, L_max, 500)
    flat_samples = np.load(f'../curve_fit/chains/mcmc_schechter_fit_chain_nb{nb1}-{nb2}.npy')
    int_list = []
    for ii, step in enumerate(flat_samples[::-1]):
        if ii == 1_000:
            break
        Phi_fit_i = schechter(Lx, 10**step[0], 10**step[1], step[2]) * Lx * np.log(10)
        int_list.append(trapezoid(Phi_fit_i * Lx, np.log10(Lx)))

    LF_int_list.append(np.median(int_list))
    LF_int_err_up_list.append(np.percentile(int_list, 84) - np.median(int_list))
    LF_int_err_down_list.append(-np.percentile(int_list, 16) + np.median(int_list))



z_list = []
# this_nb_list = [[ii, ii] for ii in range(19)]
this_nb_list = nb_fit_list
for [nb1, nb2] in this_nb_list:
    z_min = (w_central[nb1] - fwhm_Arr[nb1] * 0.5) / w_lya - 1
    z_max = (w_central[nb2] + fwhm_Arr[nb2] * 0.5) / w_lya - 1
    z_list.append([z_min, z_max])

z_central_list = [(zmax + zmin) * 0.5 for [zmin, zmax] in z_list]


fig, ax = plt.subplots(figsize=(6, 5))

ax.errorbar(z_central_list[:-1], LF_int_list[:-1],
            yerr=[np.array(LF_int_err_down_list[:-1]),
                  np.array(LF_int_err_up_list[:-1])],
            fmt='o', ms=8, capsize=2,
            ecolor='k', mfc='limegreen', mec='k', mew=1,
            label=r'This work', zorder=99)


# ax.errorbar(z_central_list[-1], LF_int_list[-1] - LF_int_err_down_list[-1],
ax.errorbar(4.75, 1.05e37,
            # yerr=LF_int_err_up_list[-1]*2,
            yerr=1.05e37,
            fmt='o', ms=8, capsize=2,
            ecolor='k', mfc='limegreen', mec='k', mew=1.1,
            lolims=True, zorder=99)


# Fit to a power-law
def linear_fit(x, a, b):
    return b + a * np.array(x)
def pl_fit(x, a, b):
    return np.power(10., linear_fit(x, a, b))

LF_int_err_list = (np.array(LF_int_err_down_list)**2 + np.array(LF_int_err_up_list)**2)**0.5
LF_int_err_list = np.array(LF_int_err_list) / np.array(LF_int_list)
popt, pcov = curve_fit(linear_fit, z_central_list[:-1], np.log10(LF_int_list)[:-1],
                       sigma=LF_int_err_list[:-1],
                       p0=[-0.5, 40])
z_xx = np.linspace(1.8, 6, 10)
n_samples = 1000
random_samples = np.random.multivariate_normal([*popt], pcov, n_samples)
line_fits = []
for i in range(n_samples):
    [a_fit, b_fit] = random_samples[i]
    line_fits.append(pl_fit(z_xx, a_fit, b_fit))
fit_16, fit_50, fit_84 = np.percentile(line_fits, [16, 50, 84], axis=0)

ax.fill_between(z_xx, fit_16, fit_84,
                color='darkgreen', alpha=0.3, lw=0,
                zorder=94)
ax.plot(z_xx, pl_fit(z_xx, *popt),
        ls='--', lw=2.5, c='darkgreen', zorder=95,
        label='Power-law fit')



print(f'{popt=}')
print(f'{pcov=}')


# Load miniJPAS data
z_central_list = np.load('/home/alberto/cosmos/LAEs/npy/z_central_list.npy')
# LF_L_int = np.load('/home/alberto/cosmos/LAEs/npy/LF_L_int_list.npy')
# LF_L_int_yerr = np.load('/home/alberto/cosmos/LAEs/npy/LF_L_int_yerr_list.npy')
# LF_int_xerr = np.load('/home/alberto/cosmos/LAEs/npy/LF_int_xerr_list.npy')
LF_L_int = [5.9789205979033454e+38, 7.922545571264946e+38, 1.0137057299239579e+39, 3.908506630437591e+38, 4.166235182624456e+38, 5.480821217451217e+38]
LF_L_int_err_up = [2.9171701671427433e+38, 4.211001696516928e+38, 1.2010272093714002e+39, 1.3792669541890399e+38, 3.651911063990588e+38, 2.0621849124720384e+38]
LF_L_int_err_down = [1.3732891411862132e+38, 2.1347054334491614e+38, 4.001750668254037e+38, 9.249679027058977e+37, 1.4081803413348322e+38, 1.267143971320615e+38]
ax.errorbar(z_central_list, LF_L_int,
            yerr=[LF_L_int_err_up, LF_L_int_err_down],
            # xerr=LF_int_xerr,
            fmt='s', ms=7, capsize=2,
            ecolor='r', mfc='r', mec='dimgray',
            label=r'Torralba-Torregrosa+23 (AGN)', zorder=-1)


# Integrate LFs of the literature
# Liu 2022
lit_ref_list = [refLF.l22_z225, refLF.l22_z237,
                refLF.l22_z254, refLF.l22_z324]
lit_z_Arr = []
lit_int_list = []
lit_int_err_list = []
L_bin_w_list = []
for lf in lit_ref_list:
    lit_z_Arr.append(lf['z'])

    this_L_bin_width = lf['logL'][1] - lf['logL'][0]
    L_bin_w_list.append(this_L_bin_width)
    mask = (lf['logL'] > L_min) & (lf['logL'] < L_max)
    lit_int_list.append((lf['Phi'] * this_L_bin_width * 10**lf['logL'])[mask].sum())
    lit_int_err_list.append((((lf['yerr_plus']) * this_L_bin_width * 10**lf['logL'])**2)[mask].sum() ** 0.5)

ax.errorbar(lit_z_Arr, lit_int_list,
            yerr=lit_int_err_list,
            fmt='X', ms=7, capsize=2,
            ecolor='orange', mfc='orange', mec='dimgray',
            label='Liu+22 (AGN)')

# Zhang 2019


# Sobral+ 2018
pathname = '/home/alberto/almacen/PAUS_data/sobral2018_rho_Lya.csv'
s18_rho_lya = pd.read_csv(pathname)

ax.errorbar(s18_rho_lya['z'], 10**s18_rho_lya['rho_Lya'],
            yerr=[10**s18_rho_lya['rho_Lya_err_up'] - 10**s18_rho_lya['rho_Lya'],
                  10**s18_rho_lya['rho_Lya'] - 10**s18_rho_lya['rho_Lya_err_down']],
            fmt='d', ms=7, capsize=2,
            ecolor='skyblue', mfc='skyblue', mec='dimgray',
            label='Sobral+18 (SFG)')

# Lin+ 2022
ax.errorbar(2.4, 6.6e40, yerr=[[3.3e40], [3.1e40]],
            fmt='*', ms=7, capsize=2, ecolor='teal',
            mfc='teal', mec='dimgray',
            label=r'Lin+22 (Diffuse Ly$\alpha$)')


ax.legend(fontsize=11, ncol=2,
          bbox_to_anchor=(1, 1.25))

ax.tick_params(labelsize=18, direction='in', which='both')
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')

ax.set_xlim(1.9, 5.5)
ax.set_ylim(1e36, 2e41)
# ax.set_ylim(2e37, 2e39)
ax.set_yscale('log')

ax.set_ylabel(r'$\rho_{\mathrm{Ly}\alpha}$ [erg\,s$^{-1}$\,Mpc$^{-3}$]', fontsize=18)
ax.set_xlabel('Redshift', fontsize=18)

fig.savefig('../figures/Integrated_Luminosity.pdf', pad_inches=0.1, bbox_inches='tight',
            facecolor='w')
plt.show()

In [ ]:
from plot.puricomp1D import plot_puricomp1d

nb_list = [[0, 3], [2, 5], [4, 7], [6, 9], [8, 11], [10, 13], [12, 15], [14, 18]]
r_min, r_max = 17, 23

fig, axs = plt.subplots(2, 1, figsize=(6, 6),
                        sharex=True)

puricomp1d_L_bins = np.linspace(17, 25, 10)
puricomp1d_L_bins_c = bin_centers(puricomp1d_L_bins)

cmap = plt.get_cmap('rainbow')

for jj, [nb_min, nb_max] in enumerate(nb_list):
    print(f'nb{nb_min}-{nb_max}')
    puri1d, comp1d = plot_puricomp1d('W1', nb_min, nb_max,
                                     r_min, r_max,
                                     L_bins=puricomp1d_L_bins,
                                     LF_kind='r')

    color = cmap(jj / len(nb_list))

    this_label = f'$z =$ {z_NB(nb_min):0.1f}-{z_NB(nb_max):0.1f}'

    axs[0].plot(puricomp1d_L_bins_c, puri1d,
                label=this_label,
                c=color, lw=2)
    axs[1].plot(puricomp1d_L_bins_c, comp1d,
                label=this_label,
                c=color, lw=2)

puri1d, comp1d = plot_puricomp1d('W1', 0, 18,
                                    r_min, r_max,
                                    L_bins=puricomp1d_L_bins,
                                    LF_kind='r')

this_label = 'Total'
color = 'k'

axs[0].plot(puricomp1d_L_bins_c, puri1d,
            label=this_label,
            c=color, lw=3, ls='--')
axs[1].plot(puricomp1d_L_bins_c, comp1d,
            label=this_label,
            c=color, lw=3, ls='--')

axs[0].set_ylabel('Purity')
axs[1].set_ylabel('Completeness')

for ax in axs:
    ax.set_ylim(0, 1)
    ax.set_xlim(24, 17)
    # ax.set_xticks([43, 43.5, 44, 44.5, 45])
    ax.tick_params(direction='in', which='both', labelsize=14)
    ax.yaxis.set_ticks_position('both')
    ax.xaxis.set_ticks_position('both')

axs[1].set_xlabel(r'$r_{\rm synth}$')
axs[1].legend(fontsize=11, ncol=2)

fig.subplots_adjust(hspace=0.1)

fig.savefig('../figures/1d_puricomp_r.pdf', pad_inches=0.1, bbox_inches='tight',
            facecolor='w')
plt.show()

In [ ]:
from plot.puricomp1D import plot_puricomp1d

# nb_list = [[0, 2], [2, 4], [4, 6], [6, 8],
#            [8, 10], [10, 12], [12, 14], [14, 16], [16, 18]]
nb_list = [[0, 3], [2, 5], [4, 7], [6, 9], [8, 11], [10, 13], [12, 15], [14, 18]]
r_min, r_max = 17, 23

fig, axs = plt.subplots(2, 1, figsize=(6, 6),
                        sharex=True)

puricomp1d_L_bins = np.linspace(42.5, 45.5, 15)
puricomp1d_L_bins_c = bin_centers(puricomp1d_L_bins)

cmap = plt.get_cmap('rainbow')

for jj, [nb_min, nb_max] in enumerate(nb_list):
    print(f'nb{nb_min}-{nb_max}')
    puri1d, comp1d = plot_puricomp1d('W3', nb_min, nb_max,
                                     r_min, r_max,
                                     L_bins=puricomp1d_L_bins)
    comp1d[puricomp1d_L_bins_c < 43.2] = 0.

    color = cmap(jj / len(nb_list))

    this_label = f'$z =$ {z_NB(nb_min):0.1f}-{z_NB(nb_max):0.1f}'

    axs[0].plot(puricomp1d_L_bins_c, puri1d,
                label=this_label,
                c=color, lw=2)
    axs[1].plot(puricomp1d_L_bins_c, comp1d,
                label=this_label,
                c=color, lw=2)

puri1d, comp1d = plot_puricomp1d('W1', 0, 18,
                                    r_min, r_max,
                                    L_bins=puricomp1d_L_bins,
                                    LF_kind='Lya')

this_label = 'Total'
color = 'k'

axs[0].plot(puricomp1d_L_bins_c, puri1d,
            label=this_label,
            c=color, lw=3, ls='--')
axs[1].plot(puricomp1d_L_bins_c, comp1d,
            label=this_label,
            c=color, lw=3, ls='--')

axs[0].set_ylabel('Purity')
axs[1].set_ylabel('Completeness')

for ax in axs:
    ax.set_ylim(0, 1)
    ax.set_xlim(42.8, 45.25)
    ax.set_xticks([43, 43.5, 44, 44.5, 45])
    ax.tick_params(direction='in', which='both', labelsize=14)
    ax.yaxis.set_ticks_position('both')
    ax.xaxis.set_ticks_position('both')

axs[1].set_xlabel(r'$\log_{10}(L_{\mathrm{Ly}\alpha}/\mathrm{erg\,s}^{-1})$')
axs[1].legend(fontsize=11, ncol=2)

fig.subplots_adjust(hspace=0.1)

fig.savefig('../figures/1d_puricomp.pdf', pad_inches=0.1, bbox_inches='tight',
            facecolor='w')
plt.show()

In [ ]:
from plot.puricomp1D import plot_puricomp1d

nb_list = [[0, 3], [2, 5], [4, 7], [6, 9], [8, 11], [10, 13], [12, 15], [14, 18]]
r_min, r_max = 17, 23

fig, axs = plt.subplots(2, 1, figsize=(6, 6),
                        sharex=True)

puricomp1d_L_bins = np.linspace(-31, -20, 7)
puricomp1d_L_bins_c = bin_centers(puricomp1d_L_bins)

cmap = plt.get_cmap('rainbow')

for jj, [nb_min, nb_max] in enumerate(nb_list):
    print(f'nb{nb_min}-{nb_max}')
    puri1d, comp1d = plot_puricomp1d('W1', nb_min, nb_max,
                                     r_min, r_max,
                                     L_bins=puricomp1d_L_bins,
                                     LF_kind='UV')
    comp1d[puricomp1d_L_bins_c > -22] = 0.

    color = cmap(jj / len(nb_list))

    this_label = f'$z =$ {z_NB(nb_min):0.1f}-{z_NB(nb_max):0.1f}'

    axs[0].plot(puricomp1d_L_bins_c, puri1d,
                label=this_label,
                c=color, lw=2)
    axs[1].plot(puricomp1d_L_bins_c, comp1d,
                label=this_label,
                c=color, lw=2)

puri1d, comp1d = plot_puricomp1d('W1', 0, 18,
                                    r_min, r_max,
                                    L_bins=puricomp1d_L_bins,
                                    LF_kind='UV')
comp1d[puricomp1d_L_bins_c > -22] = 0.

this_label = 'Total'
color = 'k'

axs[0].plot(puricomp1d_L_bins_c, puri1d,
            label=this_label,
            c=color, lw=3, ls='--')
axs[1].plot(puricomp1d_L_bins_c, comp1d,
            label=this_label,
            c=color, lw=3, ls='--')

axs[0].set_ylabel('Purity')
axs[1].set_ylabel('Completeness')

for ax in axs:
    ax.set_ylim(0, 1)
    ax.set_xlim(-21, -29)
    # ax.set_xticks([43, 43.5, 44, 44.5, 45])
    ax.tick_params(direction='in', which='both', labelsize=14)
    ax.yaxis.set_ticks_position('both')
    ax.xaxis.set_ticks_position('both')

axs[1].set_xlabel(r'$M_{\rm UV}$')
axs[1].legend(fontsize=11, ncol=2)

fig.subplots_adjust(hspace=0.1)

fig.savefig('../figures/1d_puricomp_UV.pdf', pad_inches=0.1, bbox_inches='tight',
            facecolor='w')
plt.show()